In [1]:
import cv2
import numpy as np
import imutils
import math

In [2]:
image = cv2.imread("Final.jpg")
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (7, 7), 0)

In [3]:
ret,thresh1 = cv2.threshold(gray ,127,255,cv2.THRESH_BINARY_INV)

In [4]:
dilate = cv2.dilate(thresh1, None, iterations=2)

In [5]:
cnts = cv2.findContours(dilate.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if imutils.is_cv2() else cnts[1]

In [6]:
sorted_ctrs = sorted(cnts, key=lambda ctr: cv2.boundingRect(ctr)[0] + cv2.boundingRect(ctr)[1] * image.shape[1] )

In [7]:
def calculateDistance(x1,y1,x2,y2):
    dist = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return dist

def containsContour(cnt, done):
    for x in done:
        found = np.array_equal(cnt, x)
        if found is True:
            return True
    return False

In [17]:
for cnt in sorted_ctrs:
     if(cv2.contourArea(cnt) < 400):
        sorted_ctrs.remove(cnt)

C:\Users\Intel\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
imageWidth = image.shape[1]
print(imageWidth)

3304


In [20]:
done = []
i = 0
contour = 0
curr_x1 = 0
curr_y1 = 0
while i<len(sorted_ctrs):
    
    midpoint_x1 = curr_x1
    midpoint_y1 = curr_y1
    minDistance = math.inf
    roi = 0
    
    for cnt2 in sorted_ctrs:
        found = containsContour(cnt2, done)
        if found is True:
            continue
            
        x2,y2,w2,h2 = cv2.boundingRect(cnt2)
        midpoint_x2 = x2
        midpoint_y2 = ((y2+h2)+y2)/2
        Distance = calculateDistance(midpoint_x1,midpoint_y1,midpoint_x2,midpoint_y2)
        if(Distance<minDistance): # Find minimum distance
            minDistance = Distance
            roi = image[y2:y2+h2, x2:x2+w2]
            curr_x1 = x2+w2
            curr_y1 = (int)((y2+h2)+y2)/2
            contour = cnt2
    
    # Checking left-most edge
    Distance = calculateDistance(midpoint_x1,midpoint_y1,imageWidth-1,midpoint_y1)
    if(Distance<minDistance): # Change the line
        curr_x1 = 0
        #curr_y1 = curr_y1
        continue

    if roi is 0:
        i = i + 1
        continue
    done.append(contour)
    
    # Save your contours or characters
    cv2.imwrite("Sorted-Contour/roi" + str(i) + ".png", roi)

    i = i + 1 